# Protein Domain Finder 2: End-to-End Pipeline

This notebook runs the complete pipeline for protein domain detection as implemented in the chloeguerrero62/protein-domain-finder-2 repository.

It covers:
- Downloading selected protein PDB structures
- Computing distance matrices
- Running clustering algorithms (Louvain, Two-Stage Spectral, etc.)
- Comparing all methods
- Running random controls/ablations
- Summarizing results and benchmarking
- Visualization

*Be sure to have the `src` directory and required dependencies installed! See repo README for setup instructions.*

## 1. Download Selected PDB Structures
---

This downloads all PDB files as listed in `data/pdb_clustering/selected_proteins_mmseqs2.csv` into `data/selected_structures`. Requires Biopython.

In [ ]:
import pandas as pd
from Bio.PDB import PDBList
import os

def download_selected_structures(csv_file, output_dir="data/selected_structures"):
    os.makedirs(output_dir, exist_ok=True)
    df = pd.read_csv(csv_file)
    pdbl = PDBList()
    for i, row in df.iterrows():
        pdb_id = row['pdb_id'].lower()
        print(f"  [{i+1}/{len(df)}] {pdb_id}")
        try:
            pdbl.retrieve_pdb_file(
                pdb_id,
                pdir=output_dir,
                file_format='pdb'
            )
        except Exception as e:
            print(f"    ERROR: {e}")
        
# Uncomment to run (requires Biopython and internet)
# download_selected_structures("data/pdb_clustering/selected_proteins_mmseqs2.csv")

## 2. Compute Distance Matrices

Requires `src/features/structure_parser.py` and `src/features/distance_matrix.py`. Each selected protein’s CA-CA distance matrix is computed and saved.

In [ ]:
from pathlib import Path
import sys
import numpy as np

sys.path.insert(0, str(Path('.').absolute()))
from src.features.structure_parser import ProteinStructureParser
from src.features.distance_matrix import compute_distance_matrix

# Setup output
df = pd.read_csv('data/pdb_clustering/selected_proteins_mmseqs2.csv')
output_dir = Path('data/distance_matrices')
output_dir.mkdir(parents=True, exist_ok=True)

parser = ProteinStructureParser(pdb_dir='data/selected_structures')

for i, row in df.iterrows():
    pdb_id = row['pdb_id']
    chain_id = row['chain']
    pdb_chain = f"{pdb_id}_{chain_id}"
    print(f"[{i+1}/{len(df)}] {pdb_chain}...")
    try:
        coords, res_ids = parser.parse_structure(pdb_id, chain_id)
        D = compute_distance_matrix(coords)
        np.save(output_dir / f'{pdb_id}_{chain_id}_distmat.npy', D)
        print("  ✓ success")
    except Exception as e:
        print(f"  ✗ {e}")

## 3. Run Clustering Methods

This section demonstrates unsupervised Louvain clustering and best unsupervised two-stage spectral. (See source scripts for more variants.)

*Requires:*
- `src/models/louvain_clustering.py`
- `src/features/graph_builder.py`
- `src/features/structure_parser.py`
- `src/features/distance_matrix.py`
- `src/evaluation/metrics.py`

In [ ]:
# --- Louvain Clustering Example ---
from src.models.louvain_clustering import louvain_clustering, partition_to_labels
from src.features.graph_builder import build_knn_graph
import pickle

louvain_results = {}

for i, row in df.iterrows():
    pdb_id = row['pdb_id']
    chain_id = row['chain']
    pdb_chain = f"{pdb_id}_{chain_id}"
    try:
        D = np.load(f"data/distance_matrices/{pdb_id}_{chain_id}_distmat.npy")
        G = build_knn_graph(D, k=10)
        partition = louvain_clustering(G, resolution=1.0, random_state=42)
        labels = partition_to_labels(partition)
        louvain_results[pdb_chain] = labels
        np.save(f"data/clusters/{pdb_chain}_partition.npy", labels)
        with open(f"data/clusters/{pdb_chain}_partition.pkl", 'wb') as f:
            pickle.dump(partition, f)
        print(f"Louvain done for {pdb_chain}")
    except Exception as e:
        print(f"Louvain error for {pdb_chain}: {e}")

In [ ]:
# --- Two-Stage Spectral Clustering Example (est. n_domains, then spectral) ---
from src.evaluation.metrics import compute_all_metrics
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score

def estimate_domain_count(distance_matrix, method='silhouette', max_domains=8, sigma_factor=1.0):
    sigma = np.median(distance_matrix[distance_matrix > 0]) * sigma_factor
    similarity = np.exp(-distance_matrix**2 / (2 * sigma**2))
    n_residues = len(distance_matrix)
    max_k = min(max_domains, n_residues // 10)
    best_k, best_score = 2, -1
    for k in range(2, max_k + 1):
        clustering = SpectralClustering(
            n_clusters=k,
            affinity='precomputed',
            random_state=42,
            assign_labels='kmeans'
        )
        labels = clustering.fit_predict(similarity)
        score = silhouette_score(distance_matrix, labels, metric='precomputed')
        if score > best_score:
            best_score = score
            best_k = k
    return best_k

spectral_results = {}

for i, row in df.iterrows():
    pdb_id = row['pdb_id']
    chain_id = row['chain']
    pdb_chain = f"{pdb_id}_{chain_id}"
    try:
        D = np.load(f"data/distance_matrices/{pdb_id}_{chain_id}_distmat.npy")
        n_domains = estimate_domain_count(D)
        sigma = np.median(D[D > 0])
        similarity = np.exp(-D**2 / (2 * sigma**2))
        clustering = SpectralClustering(
            n_clusters=n_domains,
            affinity='precomputed',
            random_state=42,
            assign_labels='kmeans')
        labels = clustering.fit_predict(similarity)
        spectral_results[pdb_chain] = labels
        np.save(f"data/results/two_stage_labels/{pdb_chain}_labels.npy", labels)
        print(f"Spectral done for {pdb_chain}")
    except Exception as e:
        print(f"Spectral error for {pdb_chain}: {e}")

## 4. Compare All Methods Across Proteins

This block runs all methods in a batch and aggregates detailed metrics for analysis. (Simplified demo; see repository for all metrics/functions.)

In [ ]:
# This block assumes src/evaluation/clustering_comparison.py provides: apply_louvain, apply_spectral_distance, etc.
from src.evaluation.clustering_comparison import apply_louvain, apply_spectral_distance, apply_spectral_graph, apply_hierarchical, apply_two_stage_spectral
all_results = []
parser = ProteinStructureParser(pdb_dir='data/selected_structures')

for i, row in df.iterrows():
    pdb_id = row['pdb_id']
    chain_id = row['chain']
    n_true = row['n_domains']
    try:
        coords, _ = parser.parse_structure(pdb_id, chain_id)
        D = compute_distance_matrix(coords)
        G = build_knn_graph(D, k=10)
        # Louvain
        labels = apply_louvain(D, G)
        metrics = compute_all_metrics(labels, None, n_true)
        metrics['method'] = 'Louvain'
        # Spectral (distance)
        labels = apply_spectral_distance(D, n_true)
        metrics2 = compute_all_metrics(labels, None, n_true)
        metrics2['method'] = 'Spectral-Distance'
        # (add more methods as needed)
        all_results.extend([metrics, metrics2])
    except Exception as e:
        print(f"Comparison error for {pdb_id}_{chain_id}: {e}")

# Convert to DataFrame and save
results_df = pd.DataFrame(all_results)
results_df.to_csv('data/results/method_comparison.csv', index=False)

## 5. Run Random Controls and Ablation Studies

See `scripts/random_controls.py` for control setup. Example control:

In [ ]:
def random_assignment_control(n_residues, n_domains, n_trials=10):
    errors = []
    for _ in range(n_trials):
        labels = np.random.randint(0, n_domains, size=n_residues)
        error = 0
        errors.append(error)
    return {
        'n_predicted': n_domains,
        'mean_error': np.mean(errors),
        'method': 'Random-Oracle'
    }
# Example usage for a protein:
row = df.iloc[0]
random_assignment_control(row['n_residues'] if 'n_residues' in row else 100, row['n_domains'])

## 6. Summarize Results (Tables)

Outputs main comparison/results tables as CSV and console.

Requires the results files from previous steps.

In [ ]:
method_comp = pd.read_csv('data/results/method_comparison.csv')
# Table: Per-method summary
tbl = method_comp.groupby(['method']).agg({
    'pdb_chain': 'count',
    'exact_match': 'sum',
    'absolute_error': 'mean',
    'n_predicted': 'mean'
})
tbl.columns = ['N', 'Exact Matches', 'MAE', 'Avg Predicted']
tbl['Exact Match %'] = (tbl['Exact Matches'] / tbl['N'] * 100).round(1)
print(tbl)
tbl.to_csv('data/results/method_summary_table.csv')

## 7. Visualize Example Domain Assignments

Visualize results for a given protein using matplotlib.

In [ ]:
import matplotlib.pyplot as plt

# Example: visualize Louvain assignments for one protein
pdb_chain = list(louvain_results.keys())[0]
labels = louvain_results[pdb_chain]
plt.figure(figsize=(10, 2))
plt.scatter(range(len(labels)), labels, c=labels, cmap='tab20')
plt.title(f'Louvain clustering: {pdb_chain}')
plt.xlabel('Residue Position')
plt.ylabel('Domain Cluster')
plt.show()